# Average age at infection in the SIR model
Continuing our investigation of the Susceptible-Infected-Recovered model, we will add demographics and investigate the behavior of the model in and around the endemic equilibrium. 

$$
\dot{S} = -\frac{\beta*S*I}{N} + \mu N - \mu S\\

\dot{I} = \frac{\beta*S*I}{N} - \gamma I - \mu I\\

\dot{R} = \gamma I - \mu R
$$

Analysis of this system can be found in other sources (ref. Keeling/Rohani).  Setting the derivatives to zero and doing a bit of algebra gets you to the endemic equilibrium.

$$
(S^*, \: I^*, \: R^*) = (\frac{1}{R_0}, \:\: \frac{\mu (R_0-1)}{\beta}, \: \: 1-\frac{1}{R_0} - \frac{\mu (R_0-1)}{\beta}) \\

\text{where} \:\: R_0 = \frac{\beta}{\gamma + \mu}
$$

An individual susceptible's mean time to infection will then be the inverse of the total force of infection, β<sup>I</sup>. 
$$
\tau_{S \rightarrow I} = \frac{1}{\mu (R_0 -1)}
$$

The equation above is often also described as the average age at infection.  However, it is key to remember that the measured age at infection will be censored by non-disease deaths occurring at a rate μ.  You can work through the impact of this, but in the simple case of constant mortality, it turns out to exactly balance out the *-1* in the above equation, and so our *observed* average age at infection will be *1/μR<sub>0</sub>*.  Since the hazard is constant at equilibrium, this implies an exponential distribution:

$$P(a | S->I) \sim \mu R_0 e^{-\mu R_0 a}$$

Alternatively, we can correct for the influence of mortality by observing the fraction of children who are susceptible at age A, which necessarily conditions on survival to age A.  This will look like the cumulative distribution of an exponential distributed according to the mean time to infection above:

$$P(R | a) \sim 1 - e^{(-a \mu (R_0-1))}$$

### Construct the model
In the first few cells, we do all the necessary imports.  Because we are now measuring an equilibrium property of this system, we will have to burn in, with repeated infection importations to ensure that we establish an endemic equilibrium.  We also need to model a large enough population to sustain the infection (see Critical Community Size notebook).  The primary determinant of the critical community size is the duration of infection, so keeping this relatively long (2 months) prevents us from needing huge agent populations.  So, we construct a single-patch LASER model with five components: `Births_ConstantPop`, `Susceptibility`, `Transmission`, `Infection`, and `Infect_Random_Agents`, a function that implements repeated importation for a fixed period of time. 

### Sanity check
The first test, as always, ensures that certain basic constraints are being obeyed by the model.  As it stands, I am not actually explicitly tracking the recovered population, but I can check that 
$$S_t = N_t - \sum{\Delta_I}$$.  

### Scientific test
The scientific test will sample a set of (μ, γ, R<sub>0</sub>) pairs and confirm that both the age at infection and the fraction of susceptibles at a given age are well-described by exponential distributions and that the associated rate constant is as expected.   

### Future work
The addition of an exposed compartment should not change this result (other than changing the relevant transition from S->E rather than S->I) and we should test this.  



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from laser_core.propertyset import PropertySet
from scipy.optimize import curve_fit
from scipy.stats import expon
from scipy.stats import kstest
from sklearn.metrics import mean_squared_error

from laser_generic import Births_ConstantPop
from laser_generic import Infection
from laser_generic import Model
from laser_generic import Susceptibility
from laser_generic import Transmission
from laser_generic.importation import Infect_Random_Agents
from laser_generic.utils import seed_infections_randomly
from laser_generic.utils import set_initial_susceptibility_randomly

f"{np.__version__=}"

In [ ]:
scenario = pd.DataFrame(data=[["homenode", 1e5, "47°36′35″N 122°19′59″W"]], columns=["name", "population", "location"])
parameters = PropertySet(
    {"seed": 4, "nticks": 18250, "verbose": True, "beta": 0.1, "inf_mean": 60, "cbr": 90, "importation_period": 180, "importation_count": 3}
)

In [ ]:
model = Model(scenario, parameters)
model.components = [
    Infect_Random_Agents,
    Births_ConstantPop,
    Susceptibility,
    Infection,
    Transmission,
]

seed_infections_randomly(model, ninfections=1)
model.run()
plt.plot(model.patches.cases)

### Sanity checks
As always, check that we haven't broken anything - S+I+R = N at all times.

In [ ]:
cases = np.squeeze(model.patches.cases)
susc = np.squeeze(model.patches.susceptibility)
rec = np.squeeze(model.patches.recovered)
inc = np.squeeze(model.patches.incidence)
births = np.squeeze(model.patches.births)
pops = np.squeeze(model.patches.populations)[:-1]

print("S+I+R = N:  " + str(np.isclose(cases + susc + rec, pops).all()))

### First scientific test check
As usual, we will test this on a single simulation instance and demonstrate the analysis, before moving on to testing over a range of input values.  
Here, we are interested in equilibrium behavior, so I first place a cut to exclude all infections that occur before day 10000. We then plot the data, an exponential distribution with the expected value of 
$$\frac{1}{R_0 \mu}$$ 
a best-fit exponential distribution to the data, and perform a KS test. 

In [ ]:
mu = (1 + model.params.cbr / 1000) ** (1 / 365) - 1
R0 = model.params.beta / (1 / model.params.inf_mean + mu)
A = 1 / (R0 * mu) / 365
a = np.arange(0, 15, 100)


def fit_age_at_infection(model, cutpos=10000):
    cut = model.population.doi > cutpos
    data = (model.population.doi[cut] - model.population.dob[cut]) / 365
    expfit = expon.fit(data)
    fitqual = kstest(data, expon.cdf, expfit)
    return data, expfit, fitqual


data, expfit, fitqual = fit_age_at_infection(model)


plt.hist(data, bins=a, density=True)
plt.plot(a, 1 / A * np.exp(-a / A), "-", lw=4)
plt.plot(a, expon.pdf(a, scale=expfit[1]), "k--")
plt.xlabel("Age at infection (years)")
plt.ylabel("Density")
plt.legend(
    [
        f"Expected exponential distribution - A = {A:.2f} years",
        f"Best fit age of infection, A = {expfit[1]:.2f} years",
        "Ages from simulation",
    ]
)
plt.show()
fitqual

### Second scientific test check
As noted above, because of the censoring that mortality induces, the distributions of age at infection vs. fraction susceptible at a given age will both follow exponential behavior but with a slight difference in the mean of the distribution - for the fraction susceptible, that value will be 
$$\frac{1}{\mu (R_0-1)}$$.  
We develop this second test below. 

In [ ]:
A2 = 1 / ((R0 - 1) * mu) / 365


def Scurve(x, lam):
    return np.exp(-x / lam)


def fit_susceptibility_vs_age(model, cutpos=10000):
    cut = (model.population.susceptibility == 1) & (model.population.dob > cutpos)
    bins = np.linspace(0, 50, 1200)
    age_years = (np.max(model.population.dob) - model.population.dob) / 365
    n1 = np.histogram(age_years, bins=bins)[0]
    n2 = np.histogram(age_years[cut], bins=bins)[0]
    n1[n1 == 0] = 1
    y1 = n2 / n1
    popt, pcov = curve_fit(Scurve, bins[:-1] + np.mean(np.diff(bins)), y1, p0=2.5)
    mse = mean_squared_error(y1, Scurve(bins[:-1] + np.mean(np.diff(bins)) / 2, *popt))
    return y1, bins, popt, pcov, mse


data, bins, popt, pcov, mse = fit_susceptibility_vs_age(model)

# Plotting is a little more complicated here, want to plot at the bin centers and force y=1 at x=0
plt.plot([0, *bins[:-1] + np.mean(np.diff(bins)) / 2], [1, *data], ".")
plt.plot([0, *bins[:-1] + np.mean(np.diff(bins)) / 2], [1, *(Scurve(bins[:-1] + np.mean(np.diff(bins)) / 2, A2))], lw=4)
plt.plot([0, *bins[:-1] + np.mean(np.diff(bins)) / 2], [1, *(Scurve(bins[:-1] + np.mean(np.diff(bins)) / 2, *popt))], "k--")
plt.legend(
    ["Simulation output", f"Expected exponential distribution - A = {A2:.2f} years", f"Best fit age of infection, A = {popt[0]:.2f} years"]
)
plt.xlabel("Age (years)")
plt.ylabel("Fraction of individuals susceptible at age")
plt.show()
print("RMSE = ", np.sqrt(mse))

### Third scientific test
As long as we are here, let's test that the age distribution of the population is correct.  With constant birth rate, equal to death rate, the population should have exponentially distributed ages with parameter 1/μ.

In [ ]:
# Test that population age distribution is correct
def fit_population_age_distribution(model):
    age_years = (np.max(model.population.dob) - model.population.dob) / 365
    expfit = expon.fit(age_years)
    fitqual = kstest(age_years, expon.cdf, expfit)
    return age_years, expfit, fitqual


age_years, expfit, fitqual = fit_population_age_distribution(model)
mu = (1 + model.params.cbr / 1000) ** (1 / 365) - 1
A = 1 / mu / 365
a = np.arange(0, 60, 0.1)

plt.hist(age_years, bins=a, density=True)
plt.plot(a, 1 / A * np.exp(-a / A), "-", lw=4)
plt.plot(a, expon.pdf(a, scale=expfit[1]), "k--")
plt.xlabel("Age at infection (years)")
plt.ylabel("Density")
plt.legend(
    [f"Expected exponential distribution - A = {A:.2f} years", f"Best fit age of infection, A = {expfit[1]:.2f} years", "Simulation output"]
)
plt.show()
fitqual


### Larger test suite
OK, so now we are going to replicate the above for many values of R<sub>0</sub> and cbr, as a scientific validity test.

TODO: As written, this takes a long time.  That's because the space of random values being sampled occasionally produces models where the average age at infection is 30 years old, or so.  Correspondingly, those models take a long time to equilibrate so that the tests don't look terrible fail.  So as of right now, these run for 200 years to sidestep this.  Should instead restrict the range of values - higher cbrs and R0s - so that the average age at infection is consistently lower and we can get away with running for like 30-50 years instead.  

In [ ]:
%%capture

scenario = pd.DataFrame(data=[["homenode", 3e5, "47°36′35″N 122°19′59″W"]], columns=["name", "population", "location"])

cbrs = 15 + 85 * np.random.rand(25)
inf_means = 5 + 45 * np.random.rand(25)
R0s = 1.5 + 8.5 * np.random.rand(25)
output = pd.DataFrame(data={"cbr": cbrs, "inf_mean": inf_means, "R0": R0s})
output["Average_Iage_observed"] = np.nan
output["Average_Iage_expected"] = np.nan
output["Average_Sage_observed"] = np.nan
output["Average_Sage_expected"] = np.nan
output["Average_age_expected"] = np.nan
output["Average_age_expected"] = np.nan
for index, row in output.iterrows():
    parameters = PropertySet(
        {
            "seed": 2,
            "nticks": 73000,
            "verbose": True,
            "beta": row["R0"] * (mu + 1 / row["inf_mean"]),
            "inf_mean": row["inf_mean"],
            "cbr": row["cbr"],
            "importation_period": 180,
            "importation_count": 3,
        }
    )

    model = Model(scenario, parameters)
    model.components = [
        Births_ConstantPop,
        Susceptibility,
        Transmission,
        Infection,
        Infect_Random_Agents,
    ]
    set_initial_susceptibility_randomly(model, 1 / row["R0"] + 0.05)
    seed_infections_randomly(model, ninfections=1)
    model.run()
    mu = (1 + model.params.cbr / 1000) ** (1 / 365) - 1
    _, expfit, _ = fit_age_at_infection(model, 36500)

    output.loc[index, "Average_Iage_expected"] = 1 / (row["R0"] * mu) / 365
    output.loc[index, "Average_Iage_observed"] = expfit[1]

    _, _, popt, _, _ = fit_susceptibility_vs_age(model, 36500)
    output.loc[index, "Average_Sage_expected"] = 1 / ((row["R0"] - 1) * mu) / 365
    output.loc[index, "Average_Sage_observed"] = popt[0]

    _, expfit, _ = fit_population_age_distribution(model)
    output.loc[index, "Average_age_expected"] = 1 / mu / 365
    output.loc[index, "Average_age_observed"] = expfit[1]

In [ ]:
# Plotting the expected and observed values
fig, axes = plt.subplots(3, 1, figsize=(10, 15))

# Plot for Average_Iage
axes[0].scatter(output["Average_Iage_expected"], output["Average_Iage_observed"], c="blue")
axes[0].plot(
    [output["Average_Iage_expected"].min(), output["Average_Iage_expected"].max()],
    [output["Average_Iage_expected"].min(), output["Average_Iage_expected"].max()],
    "r--",
)
axes[0].set_xlabel("Average_Iage_expected")
axes[0].set_ylabel("Average_Iage_observed")
axes[0].set_title("Average_Iage: Expected vs Observed")

# Plot for Average_Sage
axes[1].scatter(output["Average_Sage_expected"], output["Average_Sage_observed"], c="green")
axes[1].plot(
    [output["Average_Sage_expected"].min(), output["Average_Sage_expected"].max()],
    [output["Average_Sage_expected"].min(), output["Average_Sage_expected"].max()],
    "r--",
)
axes[1].set_xlabel("Average_Sage_expected")
axes[1].set_ylabel("Average_Sage_observed")
axes[1].set_title("Average_Sage: Expected vs Observed")

# Plot for Average_age
axes[2].scatter(output["Average_age_expected"], output["Average_age_observed"], c="purple")
axes[2].plot(
    [output["Average_age_expected"].min(), output["Average_age_expected"].max()],
    [output["Average_age_expected"].min(), output["Average_age_expected"].max()],
    "r--",
)
axes[2].set_xlabel("Average_age_expected")
axes[2].set_ylabel("Average_age_observed")
axes[2].set_title("Average_age: Expected vs Observed")

plt.tight_layout()
plt.show()

# Testing whether the expected and observed values are within 10% of each other
within_10_percent_Iage = np.abs(output["Average_Iage_expected"] - output["Average_Iage_observed"]) / output["Average_Iage_expected"] <= 0.1
within_10_percent_Sage = np.abs(output["Average_Sage_expected"] - output["Average_Sage_observed"]) / output["Average_Sage_expected"] <= 0.1
within_10_percent_age = np.abs(output["Average_age_expected"] - output["Average_age_observed"]) / output["Average_age_expected"] <= 0.1
# Calculate deviations
deviation_Iage = np.abs(output["Average_Iage_expected"] - output["Average_Iage_observed"]) / output["Average_Iage_expected"]
deviation_Sage = np.abs(output["Average_Sage_expected"] - output["Average_Sage_observed"]) / output["Average_Sage_expected"]
deviation_age = np.abs(output["Average_age_expected"] - output["Average_age_observed"]) / output["Average_age_expected"]

# Print average fractional deviation
print(f"Average fractional deviation for Iage: {deviation_Iage.mean()}")
print(f"Average fractional deviation for Sage: {deviation_Sage.mean()}")
print(f"Average fractional deviation for age: {deviation_age.mean()}")

# Print max deviation
print(f"Max deviation for Iage: {deviation_Iage.max()}")
print(f"Max deviation for Sage: {deviation_Sage.max()}")
print(f"Max deviation for age: {deviation_age.max()}")

# Print number of sims >5% and 10% away from expectation
print(f"Number of sims >5% away for Iage: {(deviation_Iage > 0.05).sum()}")
print(f"Number of sims >10% away for Iage: {(deviation_Iage > 0.1).sum()}")
print(f"Number of sims >5% away for Sage: {(deviation_Sage > 0.05).sum()}")
print(f"Number of sims >10% away for Sage: {(deviation_Sage > 0.1).sum()}")
print(f"Number of sims >5% away for age: {(deviation_age > 0.05).sum()}")
print(f"Number of sims >10% away for age: {(deviation_age > 0.1).sum()}")

In [ ]:
output